In [1]:
import mysql.connector
import nltk
from nltk.corpus import stopwords
from nltk import pos_tag, word_tokenize

In [2]:
# Download NLTK resources
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Hasee\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Hasee\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Hasee\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [50]:
# SQL Synonyms Dictionary
sql_synonyms = {
    'select': ['Retrieve', 'Fetch'],
    'from': ['In', 'Within'],
    'where': ['Filter', 'Condition','in'],
    'all' : ['*'],
    'group by': ['Group', 'Aggregate'],
    'having': ['Filter', 'Condition'],
    'order by': ['Sort', 'Arrange'],
    'insert into': ['Add', 'Create'],
    'values': ['Data', 'Information'],
    'update': ['Modify', 'Change'],
    'set': ['Assign', 'Update'],
    'delete from': ['Remove', 'Erase'],
    'join': ['Combine', 'Connect'],
    'inner join': ['Inner Combine', 'Inner Connect'],
    'left join': ['Left Combine', 'Left Connect'],
    'right join': ['Right Combine', 'Right Connect'],
    'full join': ['Full Combine', 'Full Connect'],
    'union': ['Combine', 'Merge'],
    'intersect': ['Common', 'Overlap'],
    'except': ['Difference', 'Subtract'],
    'create table': ['Define Table', 'Construct Table'],
    'alter table': ['Modify Table', 'Change Table'],
    'drop table': ['Remove Table', 'Delete Table'],
    'primary key': ['Main Key', 'Primary Identifier'],
    'foreign key': ['Reference Key', 'External Key'],
    'index': ['Search Key', 'Lookup'],
    'null': ['Empty', 'Missing'],
    'distinct': ['Unique', 'Different'],
    'and': ['Logical AND', 'Conjunction'],
    'or': ['Logical OR', 'Disjunction'],
    'not': ['Negation', 'Inverse'],
}

In [51]:
# Database configuration
db_config = {
    'host': 'HaseebAhmed',
    'user': 'root@localhost',
    'password': '',
    'database': 'adt_project',
}

In [52]:
table_names = ['employees']  # Add more table names as needed

In [77]:
def get_table_names(connection):
    table_names = []

    try:
        cursor = connection.cursor()
        cursor.execute("SHOW TABLES")
        tables = cursor.fetchall()
        table_names = [table[0] for table in tables]

    except mysql.connector.Error as e:
        print(f"Error fetching table names: {e}")

    return table_names

def get_table_columns(connection, table_name):
    columns = []

    try:
        cursor = connection.cursor()
        cursor.execute(f"DESCRIBE {table_name}")
        columns = [column[0] for column in cursor.fetchall()]

    except mysql.connector.Error as e:
        print(f"Error fetching columns for {table_name}: {e}")

    return columns

def generate_sql_query(filtered_tokens, table_names, table_info):
    # Default values
    select_clause = "SELECT *"
    from_clause = ""
    where_clause = ""

    # Identify keywords in filtered tokens
    for i, token in enumerate(filtered_tokens):
        # Use the table names to dynamically determine SQL components
        if token.lower() in table_names:
            from_clause = f"FROM {token.lower()}"
        elif token.lower() in table_info:
            # Check if the next token is "from" and the token after that is a column name
            if i + 2 < len(filtered_tokens) and filtered_tokens[i + 1].lower() == 'from' and filtered_tokens[i + 2].lower() in table_info[token.lower()]:
                column_name = filtered_tokens[i + 2]
                select_clause = f"SELECT {column_name}"
                break

    # Construct the final query
    final_query = f"{select_clause} {from_clause} {where_clause}"

    return final_query

try:
    # Connect to the database
    connection = mysql.connector.connect(**db_config)

    if connection.is_connected():
        print(f"Connected to MySQL database: {db_config['database']}")

        # Get table names
        table_names = get_table_names(connection)
        print("Table Names:")
        print(table_names)

        # Get table information
        table_info = {}
        for table_name in table_names:
            table_info[table_name] = get_table_columns(connection, table_name)
        print("Table Information:")
        print(table_info)

        # Take user input
        user_input = input("Enter a filtered query: ")

        # Tokenize and apply POS tagging
        filtered_tokens = word_tokenize(user_input)

        # Remove stop words
        stop_words = set(stopwords.words('english'))
        filtered_tokens = [word for word in filtered_tokens if word.lower() not in stop_words]

        # Generate SQL query
        generated_query = generate_sql_query(filtered_tokens, table_names, table_info)
        print("Generated SQL Query:")
        print(generated_query)

        # Execute the SQL query
        cursor = connection.cursor(dictionary=True)
        cursor.execute(generated_query)

        # Fetch and display the results
        results = cursor.fetchall()
        if results:
            print("Query Results:")
            for row in results:
                print(row)
        else:
            print("No results found.")

except mysql.connector.Error as e:
    print(f"Error: {e}")

finally:
    # Close the database connection in the finally block to ensure it always happens
    if 'connection' in locals():
        connection.close()
        print("Connection closed.")

Connected to MySQL database: adt_project
Table Names:
['employees', 'students', 'users']
Table Information:
{'employees': ['name', 'id', 'email', 'department'], 'students': ['name', 'email', 'sid'], 'users': ['id', 'username', 'email']}
Enter a filtered query: show id from users
Generated SQL Query:
SELECT * FROM users 
Query Results:
{'id': 1, 'username': 'Haseeb', 'email': 'ahmad99@uwindsor.ca'}
Connection closed.


In [76]:
def get_table_names(connection):
    table_names = []

    try:
        cursor = connection.cursor()
        cursor.execute("SHOW TABLES")
        tables = cursor.fetchall()
        table_names = [table[0] for table in tables]

    except mysql.connector.Error as e:
        print(f"Error fetching table names: {e}")

    return table_names

def get_table_columns(connection, table_name):
    columns = []

    try:
        cursor = connection.cursor()
        cursor.execute(f"DESCRIBE {table_name}")
        columns = [column[0] for column in cursor.fetchall()]

    except mysql.connector.Error as e:
        print(f"Error fetching columns for {table_name}: {e}")

    return columns

def generate_sql_query(filtered_tokens, table_names, table_info):
    # Default values
    select_clause = "SELECT *"
    from_clause = ""
    where_clause = ""

    # Identify keywords in filtered tokens
    for i, token in enumerate(filtered_tokens):
        # Use the table names to dynamically determine SQL components
        if token.lower() in table_names:
            from_clause = f"FROM {token.lower()}"
        elif token.lower() in table_info:
            # Check if the next token is "from" and the token after that is a column name
            if i + 2 < len(filtered_tokens) and filtered_tokens[i + 1].lower() == 'from' and filtered_tokens[i + 2].lower() in table_info[token.lower()]:
                column_name = filtered_tokens[i + 2]
                select_clause = f"SELECT {column_name}"
                break

    # Construct the final query
    final_query = f"{select_clause} {from_clause} {where_clause}"

    return final_query

try:
    # Connect to the database
    connection = mysql.connector.connect(**db_config)

    if connection.is_connected():
        print(f"Connected to MySQL database: {db_config['database']}")

        # Get table names
        table_names = get_table_names(connection)
        print("Table Names:")
        print(table_names)

        # Get table information
        table_info = {}
        for table_name in table_names:
            table_info[table_name] = get_table_columns(connection, table_name)
        print("Table Information:")
        print(table_info)

        # Take user input
        user_input = input("Enter a filtered query: ")

        # Tokenize and apply POS tagging
        filtered_tokens = word_tokenize(user_input)

        # Remove stop words
        stop_words = set(stopwords.words('english'))
        filtered_tokens = [word for word in filtered_tokens if word.lower() not in stop_words]

        # Generate SQL query
        generated_query = generate_sql_query(filtered_tokens, table_names, table_info)
        print("Generated SQL Query:")
        print(generated_query)

        # Execute the SQL query
        cursor = connection.cursor(dictionary=True)
        cursor.execute(generated_query)

        # Fetch and display the results
        results = cursor.fetchall()
        if results:
            print("Query Results:")
            for row in results:
                print(row)
        else:
            print("No results found.")

except mysql.connector.Error as e:
    print(f"Error: {e}")

finally:
    # Close the database connection in the finally block to ensure it always happens
    if 'connection' in locals():
        connection.close()
        print("Connection closed.")

Connected to MySQL database: adt_project
Table Names:
['employees', 'students', 'users']
Table Information:
{'employees': ['name', 'id', 'email', 'department'], 'students': ['name', 'email', 'sid'], 'users': ['id', 'username', 'email']}
Enter a filtered query: show the id from users
Generated SQL Query:
SELECT * FROM users 
Query Results:
{'id': 1, 'username': 'Haseeb', 'email': 'ahmad99@uwindsor.ca'}
Connection closed.
